# Day 24: It Hangs in the Balance

[*Advent of Code 2015 day 24*](https://adventofcode.com/2015/day/24) and [*solution megathread*](https://www.reddit.com/3y1s7f)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2015/24/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2015%2F24%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')


%load_ext nb_mypy
%nb_mypy On

Version 1.0.4


In [2]:
import common


downloaded = common.refresh()
%store downloaded >downloaded

%load_ext pycodestyle_magic
%pycodestyle_on

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [3]:
from IPython.display import HTML


HTML(downloaded['part1'])

## Comments

Cleaning this up now (purging all cruft from previous commit) - a `FrozenSet` is immutable and thus hashable, important for how we'll use it

In [4]:
from typing import FrozenSet, Tuple

PackageGroup = FrozenSet[int]
PackageConfiguration = Tuple[PackageGroup, ...]

In [5]:
testdata_packages = frozenset([1, 2, 3, 4, 5, 7, 8, 9, 10, 11])
inputdata = frozenset(int(p) for p in downloaded['input'].splitlines())

In [6]:
from typing import Iterable
from functools import reduce
import operator


def target_weight(packages: Iterable[int],
                  compartments: int = 3) -> int:
    total_weight = sum(packages)
    if not total_weight % compartments == 0:
        raise ValueError(f'weight of packages is not divisible by ' +
                         f'{compartments}: {total_weight} ({packages})')
    return total_weight // compartments


def calculate_QE(packages: Iterable[int]) -> int:
    return reduce(operator.mul, packages)


def group_str(group: PackageGroup) -> str:
    return str(set(group))


def configuration_str(configuration: PackageConfiguration) -> str:
    return '; '.join(group_str(group) for group in configuration)

In [7]:
from itertools import combinations


def minimal_combinations(packages: PackageGroup,
                         compartments: int = 3) -> \
        FrozenSet[PackageGroup]:
    # Might as well output a set rather than Iterable
    # here, because when we find a valid minimum length
    # we output all valid combinations
    group_weight = target_weight(packages, compartments)
    for length in range(
            1, len(packages) - (compartments - 2)):
        output: FrozenSet[PackageGroup] = \
            frozenset(
                frozenset(pg)
                for pg in combinations(packages, length)
                if sum(pg) == group_weight)
        if output:
            return output
    raise ValueError('could not locate any combination for ' +
                     f'{group_weight} in {packages}')

In [8]:
def best_minimal_combination(packages: PackageGroup,
                             compartments: int = 3) -> \
        PackageGroup:
    return min(minimal_combinations(packages, compartments),
               key=lambda pc: calculate_QE(pc))

In [9]:
from typing import Set


def remaining_configurations(
        packages: PackageGroup,
        previous_configurations: PackageConfiguration,
        compartments: int = 3) -> \
        Iterable[PackageConfiguration]:
    cache: Set[PackageGroup] = set()
    group_weight = sum(previous_configurations[0])
    remaining_groups = compartments - len(previous_configurations)
    remaining_packages = reduce(operator.sub,
                                previous_configurations,
                                packages)
    if remaining_groups == 1:
        assert sum(remaining_packages) == group_weight
        yield *previous_configurations, remaining_packages
    else:
        for length in range(
                1,
                len(remaining_packages) - remaining_groups + 2):
            for group in combinations(remaining_packages, length):
                if sum(group) != group_weight:
                    continue
                group_fs = frozenset(group)
                if group_fs in cache:
                    continue
                cache.add(group_fs)
                for configuration in remaining_configurations(
                        packages,
                        (*previous_configurations, group_fs),
                        compartments):
                    yield configuration

In [10]:
def all_best_configurations(packages: PackageGroup,
                            compartments: int = 3) -> \
        Iterable[PackageConfiguration]:
    passenger = best_minimal_combination(packages, compartments)
    for configuration in remaining_configurations(packages,
                                                  (passenger, ),
                                                  compartments):
        yield configuration

In [11]:
from IPython.display import display


for configuration in all_best_configurations(testdata_packages, 3):
    display(configuration_str(configuration))
    assert calculate_QE(configuration[0]) == 99
    break

'{9, 11}; {8, 2, 10}; {1, 3, 4, 5, 7}'

In [12]:
for configuration in all_best_configurations(inputdata, 3):
    display(configuration_str(configuration) +
            f' {calculate_QE(configuration[0])}')
    # This one actually do yield too many combinations
    # to manage - better to break here rather than crash
    break

'{1, 113, 103, 107, 109, 79}; {89, 97, 2, 67, 83, 101, 73}; {3, 5, 71, 7, 13, 17, 19, 23, 29, 31, 37, 41, 43, 53, 59, 61} 10723906903'

In [13]:
HTML(downloaded['part1_footer'])

## Part Two

In [14]:
HTML(downloaded['part2'])

In [15]:
for configuration in all_best_configurations(testdata_packages, 4):
    display(configuration_str(configuration))
    assert calculate_QE(configuration[0]) == 44
    break

'{11, 4}; {10, 5}; {8, 7}; {1, 2, 3, 9}'

In [16]:
for configuration in all_best_configurations(inputdata, 4):
    display(configuration_str(configuration) +
            f' {calculate_QE(configuration[0])}')
    # This one actually do yield too many combinations
    # to manage - better to break here rather than crash
    break

'{113, 59, 109, 103}; {97, 107, 101, 79}; {89, 1, 67, 83, 71, 73}; {2, 3, 5, 37, 7, 41, 43, 13, 17, 19, 61, 53, 23, 29, 31} 74850409'

In [17]:
HTML(downloaded['part2_footer'])